In [1]:
import pandas as pd
from collections import Counter
from itertools import chain
import pm4py

In [2]:
df_verslagen_clean = pd.read_csv('a:/df_verslagen_cleaned.csv')
df_verslagen_clean.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,verslagen_report_content_cleaned,content_words,content_words_lemmatized
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,dhr aj dingemans huisarts streetname nr city d...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...","['dhr', 'aj', 'dingemans', 'huisarts', 'street..."
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,samenvatting rectaal bloedverlies obv divertik...,"['samenvatting', 'rectaal', 'bloedverlies', 'o...","['samenvatting', 'rectaal', 'bloedverlie', 'ob..."
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,coloscopie betreft mw initials lastname adresg...,"['coloscopie', 'betreft', 'mw', 'initials', 'l...","['coloscopie', 'betreffen', 'mw', 'initials', ..."
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,gastroscopie betreft mw initials lastname adre...,"['gastroscopie', 'betreft', 'mw', 'initials', ...","['gastroscopie', 'betreffen', 'mw', 'initials'..."
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,samenvatting rectaal bloedverlies eenmalig hd ...,"['samenvatting', 'rectaal', 'bloedverlies', 'e...","['samenvatting', 'rectaal', 'bloedverlie', 'ee..."


In [3]:
df_verslagen = pd.read_csv('a:/bloeding-met-patientenlijst-2/bloeding-met-patientenlijst-4-verslagen.csv')

df_verslagen = df_verslagen.drop(columns=['verslagen_report_content'])
df_verslagen.head()

,pseudo_id,verslagen_report_tags,verslagen_report_start_date
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,2020-11-26 15:06:00
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",2020-11-26 09:53:00
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,2020-11-25 14:13:00
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,2020-11-25 13:48:00
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",2020-11-25 08:47:00


In [4]:
# Step 1: Sort by Patient_ID and Timestamp
df_verslagen = df_verslagen.sort_values(by=['pseudo_id', 'verslagen_report_start_date'])

# Step 2: Create a list of sequences for each patient
patient_sequences = df_verslagen.groupby('pseudo_id')['verslagen_report_tags'].apply(lambda x: '->'.join(x))

# Step 3: Count the occurrences of each unique sequence
sequence_counts = Counter(patient_sequences)

# Step 4: Convert to DataFrame for better visualization
sequence_df = pd.DataFrame(sequence_counts.items(), columns=['Sequence', 'Count'])

# Step 5: Sort by most frequent sequences
sequence_df = sequence_df.sort_values(by='Count', ascending=False)

print(sequence_df)


                                              Sequence  Count
0    Consult, SEH->Consult, SEH->Consult, SEH->Cons...      1
1    Consult, SEH->Consult, SEH->Consult, Kliniek: ...      1
2    Consult, SEH->Spoedeisende Hulp->Consult, Klin...      1
3    Consult, SEH->Consult, SEH->Consult, SEH->Cons...      1
4    Consult, SEH->Consult->Consult->Consult, Overi...      1
..                                                 ...    ...
100  Consult, SEH->Consult, SEH->Consult, SEH->Cons...      1
101  Consult, SEH->Consult, SEH->Consult, Overige a...      1
102  Consult, SEH->Consult, SEH->Consult, SEH->Cons...      1
103  Consult, SEH->Consult, SEH->Consult, Kliniek: ...      1
104  Consult, SEH->Consult, SEH->Consult, SEH->Cons...      1

[105 rows x 2 columns]


In [5]:
# Flatten all steps into a single list
all_steps = list(chain(*df_verslagen.groupby('pseudo_id')['verslagen_report_tags'].apply(list)))

# Count step frequency
step_counts = Counter(all_steps)
print(step_counts)


Counter({'Consult, Kliniek: vervolgconsult': 2249, 'Consult, Polikliniek: vervolgconsult': 1993, 'Consult': 1577, 'Consult, SEH': 1227, 'Consult, Overige aantekeningen': 682, 'Poliklinische Brief': 481, 'Consult, Telefonisch consult': 452, 'Klinische Brief': 318, 'Spoedeisende Hulp': 250, 'Consult, Kliniek: eerste consult': 139, 'Consult, Polikliniek: eerste consult': 128, 'Consult, Samenvatting': 16, 'Consult, Thuisbehandeling': 14, 'Algemeen': 14, 'Consult, Intercollegiaal consult': 12, 'Consult, Multidisciplinair overleg': 11, 'Consult, Dagbehandeling': 9, 'Radiologieverslag, ECG': 1, 'Consult, Research': 1, 'Radiologieverslag, ECG kliniek': 1, 'Radiologieverslag, Heup links': 1, 'Radiologieverslag, CT colonografie': 1})


In [6]:
# pip install pm4py

In [7]:
# Convert DataFrame to PM4Py event log
event_log = pm4py.format_dataframe(df_verslagen, case_id='pseudo_id', activity_key='verslagen_report_tags', timestamp_key='verslagen_report_start_date')

print(event_log.head())  # Verify format

                                  pseudo_id verslagen_report_tags  \
0  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6          Consult, SEH   
1  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6          Consult, SEH   
2  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6          Consult, SEH   
3  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6          Consult, SEH   
4  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6          Consult, SEH   

  verslagen_report_start_date                         case:concept:name  \
0   2013-10-21 10:24:00+00:00  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
1   2013-10-21 10:40:00+00:00  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
2   2013-10-21 10:41:00+00:00  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
3   2013-10-21 10:50:00+00:00  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
4   2013-10-22 19:59:00+00:00  046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   

   concept:name            time:timestamp  @@index  @@case_index  
0  Consult, SEH 2013-10-21 10:24:00+00:00        0             0  


In [8]:
import pm4py

log = event_log
# Discover Petri net using the inductive miner
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log)
# Visualize the Petri net
pm4py.view_petri_net(net, initial_marking, final_marking)


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH